In [1]:
!pip install datasets
!pip install evaluate
!pip install transformers

In [ ]:
from transformers import AutoProcessor, AutoModelForAudioClassification

processor = AutoProcessor.from_pretrained("mclemcrew/distilhubert-finetuned-audio-effect-classification")
model = AutoModelForAudioClassification.from_pretrained("mclemcrew/distilhubert-finetuned-audio-effect-classification")

In [22]:
from datasets import load_dataset
music_mixing = load_dataset("mclemcrew/mixology-tracks", "default")

In [23]:
music_mixing

DatasetDict({
    train: Dataset({
        features: ['track-name', 'track-type', 'channel-mode', 'track-instrument-type', 'track-instrument-subtype', 'resampled_audio', 'parameters', 'track-audio-features'],
        num_rows: 2345
    })
})

In [24]:
music_mixing = music_mixing['train']

In [15]:
music_mixing[0]

{'track-name': 'Kick In',
 'track-type': 'AUDIO',
 'channel-mode': 'MONO',
 'track-instrument-type': 'DRUMS',
 'track-instrument-subtype': 'KICK',
 'resampled_audio': [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
 

In [26]:
import json
effects = ['eq', 'compression', 'reverb', 'delay', 'gate', 'limiter', 'phaser', 'flanger', 'chorus']

def process_parameters_cell(examples):
    effect_codes = []
    for cell in examples['parameters']:
        effect_code = 0  # Start with no effects
        parameters_dict = cell
        parameters_dict = parameters_dict.replace("\'", "\"")
        for i, effect in enumerate(effects):
            if json.loads(parameters_dict).get(effect):
                effect_code |= (1 << i)  # Set the corresponding bit
        effect_codes.append(effect_code)
    return {'label': effect_codes}

# Assuming your dataset is named 'track_list_dataset'
music_mixing = music_mixing.map(process_parameters_cell, batched=True)

In [27]:
unique_labels = music_mixing.unique("label")
print(unique_labels)
print(len(unique_labels))

[16, 5, 1, 3, 0, 7, 6, 17, 4, 13, 2, 23, 19, 20, 15, 259, 21, 8, 151, 135, 33, 35, 9, 267, 71, 47, 18, 11, 32, 69, 271, 12, 27, 51, 55, 53, 263, 36, 37, 14, 24, 39, 132, 65]
44


In [34]:
from collections import Counter

def decode_effect_code(effect_code):
    effect_list = []
    for i, effect in enumerate(effects):
        if effect_code & (1 << i):  # Check if the bit is set
            effect_list.append(effect)
    return effect_list

# Assuming your music_mixing dataset has the label column
label_counts = Counter(music_mixing['label'])
decoded_label_counts = {tuple(decode_effect_code(code)): count for code, count in label_counts.items()}

print("Effect Combinations and Counts:")
for effect_combo, count in decoded_label_counts.items():
    print(f"- {', '.join(effect_combo)}: {count} times")


Effect Combinations and Counts:
- gate: 6 times
- eq, reverb: 204 times
- eq: 283 times
- eq, compression: 342 times
- : 348 times
- eq, compression, reverb: 457 times
- compression, reverb: 89 times
- eq, gate: 15 times
- reverb: 195 times
- eq, reverb, delay: 17 times
- compression: 76 times
- eq, compression, reverb, gate: 84 times
- eq, compression, gate: 71 times
- reverb, gate: 3 times
- eq, compression, reverb, delay: 55 times
- eq, compression, chorus: 1 times
- eq, reverb, gate: 14 times
- delay: 2 times
- eq, compression, reverb, gate, flanger: 1 times
- eq, compression, reverb, flanger: 1 times
- eq, limiter: 11 times
- eq, compression, limiter: 4 times
- eq, delay: 6 times
- eq, compression, delay, chorus: 3 times
- eq, compression, reverb, phaser: 3 times
- eq, compression, reverb, delay, limiter: 2 times
- compression, gate: 7 times
- eq, compression, delay: 9 times
- limiter: 4 times
- eq, reverb, phaser: 2 times
- eq, compression, reverb, delay, chorus: 3 times
- reverb

| Effect Combination                             | Count |
|------------------------------------------------|-------|
| eq, compression, reverb                        | 457   |
| no effect                                                | 348   |
| eq, compression                                | 342   |
| eq                                             | 283   |
| eq, reverb                                     | 204   |
| reverb                                         | 195   |
| compression, reverb                            | 89    |
| eq, compression, reverb, gate                  | 84    |
| compression                                    | 76    |
| eq, compression, gate                          | 71    |
| eq, compression, reverb, delay                 | 55    |
| eq, reverb, delay                              | 17    |
| eq, gate                                       | 15    |
| eq, reverb, gate                               | 14    |
| eq, limiter                                    | 11    |
| eq, compression, delay                         | 9     |
| compression, gate                              | 7     |
| gate                                           | 6     |
| eq, delay                                      | 6     |
| reverb, limiter                                | 6     |
| eq, compression, limiter                       | 4     |
| limiter                                        | 4     |
| eq, compression, delay, chorus                 | 3     |
| eq, compression, reverb, phaser                | 3     |
| eq, compression, reverb, gate, limiter         | 3     |
| eq, compression, reverb, chorus                | 3     |
| eq, compression, reverb, delay, chorus         | 3     |
| reverb, gate                                   | 3     |
| reverb, delay                                  | 3     |
| eq, reverb, limiter                            | 3     |
| delay                                          | 2     |
| eq, compression, reverb, delay, limiter        | 2     |
| eq, reverb, phaser                             | 2     |
| eq, reverb, gate, limiter                      | 2     |
| eq, compression, chorus                        | 1     |
| eq, compression, reverb, gate, flanger         | 1     |
| eq, compression, reverb, flanger               | 1     |
| eq, compression, delay, gate                   | 1     |
| eq, compression, gate, limiter                 | 1     |
| compression, reverb, delay                     | 1     |
| delay, gate                                    | 1     |
| eq, compression, reverb, limiter               | 1     |
| reverb, flanger                                | 1     |
| eq, phaser                                     | 1     |

In [9]:
music_mixing_full = music_mixing.train_test_split(test_size=0.2, seed=42, shuffle=True)

In [10]:
music_mixing_full['train'][150]['label']

18

In [11]:
music_mixing_full

DatasetDict({
    train: Dataset({
        features: ['track-name', 'track-type', 'channel-mode', 'track-instrument-type', 'track-instrument-subtype', 'resampled_audio', 'parameters', 'track-audio-features', 'label'],
        num_rows: 1876
    })
    test: Dataset({
        features: ['track-name', 'track-type', 'channel-mode', 'track-instrument-type', 'track-instrument-subtype', 'resampled_audio', 'parameters', 'track-audio-features', 'label'],
        num_rows: 469
    })
})